In [14]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
from datasets import load_dataset
import torch

import json
import os
from datasets import Dataset
from transformers import AutoTokenizer

import random
from typing import List, Tuple
from unsloth import is_bfloat16_supported

from transformers import TextStreamer

from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
from transformers import TextStreamer
from unsloth.chat_templates import train_on_responses_only
from transformers import PreTrainedTokenizerBase
import re
from collections import Counter
from transformers import BitsAndBytesConfig
import numpy as np
import torch
import gc
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

In [15]:
def train_test_split(member: str, test_size: float = 0.2, seed: int = 42, data_path: str = '/work/users/s/m/smerrill/Albemarle/dataset') -> Tuple[List[dict], List[dict]]:
    """
    Splits the dataset into training and test sets. Synthetic data is always added to the training set.

    Parameters:
    - member: The name identifier for the board member.
    - test_size: Proportion of the real (non-synthetic) data to include in the test split.
    - seed: Random seed for reproducibility.
    - data_path: Base directory for the dataset files.

    Returns:
    - A tuple (train_data, test_data)
    """
    real_data, synth_data = [], []

    if member == 'kateacuff':
        real_data = np.load(os.path.join(data_path, 'kateacuff_train.npy'))
        synth_data = np.load(os.path.join(data_path, 'synth_kateacuff.npy'))
        test_data = np.load(os.path.join(data_path, 'kateacuff_test.npy'), allow_pickle=True)
        train_completion_data = np.load(os.path.join(data_path, 'kateacuff_train_completion.npy'), allow_pickle=True)

        
    elif member == 'ellenosborne':
        real_data = np.load(os.path.join(data_path, 'ellenosborne_train.npy'))
        synth_data = np.load(os.path.join(data_path, 'synth_ellenosborne.npy'))
        test_data = np.load(os.path.join(data_path, 'ellenosborne_test.npy'), allow_pickle=True)
        train_completion_data = np.load(os.path.join(data_path, 'ellenosborne_train_completion.npy'), allow_pickle=True)
        
    elif member == 'grahampaige':
        real_data = np.load(os.path.join(data_path, 'grahampaige_train.npy'))
        synth_data = np.load(os.path.join(data_path, 'synth_grahampaige.npy'))
        test_data = np.load(os.path.join(data_path, 'grahampaige_test.npy'), allow_pickle=True)
        train_completion_data = np.load(os.path.join(data_path, 'grahampaige_train_completion.npy'), allow_pickle=True)                             
        
    elif member == 'judyle':
        real_data = np.load(os.path.join(data_path, 'judyle_train.npy'))
        synth_data = np.load(os.path.join(data_path, 'synth_judyle.npy'))
        test_data = np.load(os.path.join(data_path, 'judyle_test.npy'), allow_pickle=True)
        train_completion_data = np.load(os.path.join(data_path, 'judyle_train_completion.npy'), allow_pickle=True)
        
    elif member == 'katrinacallsen':
        real_data = np.load(os.path.join(data_path, 'katrinacallsen_train.npy'))
        test_data = np.load(os.path.join(data_path, 'katrinacallsen_test.npy'), allow_pickle=True)
        train_completion_data = np.load(os.path.join(data_path, 'katrinacallsen_train_completion.npy'), allow_pickle=True)
        
    elif member == 'davidoberg':
        real_data = np.load(os.path.join(data_path, 'davidoberg_train.npy'))
        test_data = np.load(os.path.join(data_path, 'davidoberg_test.npy'), allow_pickle=True)
        train_completion_data = np.load(os.path.join(data_path, 'davidoberg_train_completion.npy'), allow_pickle=True)
        
    elif member == 'jonnoalcaro':
        real_data = np.load(os.path.join(data_path, 'jonnoalcaro_train.npy'))
        test_data = np.load(os.path.join(data_path, 'jonnoalcaro_test.npy'), allow_pickle=True)
        train_completion_data = np.load(os.path.join(data_path, 'jonnoalcaro_train_completion.npy'), allow_pickle=True)
        
    else:
        raise ValueError(f"Unknown member: {member}")

    if not 0 < test_size < 1:
        raise ValueError("test_size must be a float between 0 and 1.")

    train_data = list(real_data) + list(synth_data)
    return train_data, test_data, train_completion_data


In [16]:
model_name = "unsloth/Llama-3.3-70B-Instruct-bnb-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_data, test_data, train_completion_data = train_test_split('kateacuff')

tmp = [{"text": text} for text in train_data]
train_data = Dataset.from_list(tmp)

In [27]:
print(train_data['text'][19])

<|begin_of_text|><|start_header_id|>assistant<|end_header_id|>

kateacuff: I would agree that that the online retreat and half a day, but I would keep it open to having part to either online or with a delayed larger group. I think having all day long online, you would see people ducking under their desk. Probably I'm entering.<|eot_id|>

<|start_header_id|>user<|end_header_id|>

rossholden: I'm entering our seven of Zoom. I can tell you I'm. My attention is drifting.
katrinacallsen: I move or do we have to make a motion you just we just have to tell you.
patrickmclau: I think we just need some consensus from the board, it sounds like we might have that around a half day. zoom meeting would we want to keep the the June 5 date, which was our original date for the retreat.
jonnoalcaro: I would be in favor of keeping the June 5th date. And on the half day issue, I agree with Kate that we need to potentially set another date for a part two, because at least some of the things that I'd like 

### Add special tokens to tokenizer

In [6]:
# Extract speaker tokens
def extract_speakers(text):
    return re.findall(r"^(?:speaker \d+|[a-zA-Z0-9_]+):", text, flags=re.MULTILINE)

speaker_counter = Counter()
for sample in train_data:
    speakers = extract_speakers(sample["text"])
    speaker_counter.update(speakers)

speaker_tokens = list(speaker_counter.keys())

# Add special tokens
special_tokens = {
    "additional_special_tokens": speaker_tokens + [
        "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"
    ]
}

tokenizer.add_special_tokens(special_tokens)


def debug_tokenization(example_text: str, tokenizer: PreTrainedTokenizerBase):
    tokens = tokenizer(example_text, return_tensors="pt", add_special_tokens=False)
    input_ids = tokens["input_ids"][0]
    decoded = [tokenizer.decode([tid]) for tid in input_ids]

    print("=== Tokenized Input ===")
    for i, (tid, tok) in enumerate(zip(input_ids, decoded)):
        print(f"{i:03}: {tid.item():>5}  ->  {repr(tok)}")

debug_tokenization(train_data[0]['text'], tokenizer)

=== Tokenized Input ===
000: 128000  ->  '<|begin_of_text|>'
001: 128006  ->  '<|start_header_id|>'
002:   882  ->  'user'
003: 128007  ->  '<|end_header_id|>'
004:   271  ->  '\n\n'
005: 128256  ->  'katrinacallser:'
006:  2100  ->  ' So'
007:   374  ->  ' is'
008:   430  ->  ' that'
009:  1057  ->  ' our'
010:  2218  ->  ' target'
011:    13  ->  '.'
012:  1120  ->  ' just'
013:   520  ->  ' at'
014:   279  ->  ' the'
015:  7314  ->  ' beginning'
016:   315  ->  ' of'
017:  7552  ->  ' February'
018:    13  ->  '.'
019:   358  ->  ' I'
020:  2846  ->  "'m"
021: 20910  ->  ' wondering'
022:   422  ->  ' if'
023:   430  ->  ' that'
024:   596  ->  "'s"
025:   279  ->  ' the'
026:  2218  ->  ' target'
027:   627  ->  '.\n'
028: 128257  ->  'patrickmclaughlin:'
029:  3011  ->  ' That'
030:   574  ->  ' was'
031:   279  ->  ' the'
032:  4113  ->  ' original'
033:  2218  ->  ' target'
034:   994  ->  ' when'
035:   584  ->  ' we'
036:  1051  ->  ' were'
037:  1701  ->  ' using'
038:  6790 

### Model

In [7]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()


max_seq_length = 850

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # or bfloat16 on Ampere+
    llm_int8_enable_fp32_cpu_offload=False,  # turn this off
)

model, _ = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=None,
    load_in_4bit=True,
    load_in_8bit=False,
    device_map=None, 
    trust_remote_code=True,
    quantization_config=bnb_config
)

model.resize_token_embeddings(len(tokenizer), mean_resizing=False)

"""
model = FastLanguageModel.get_peft_model(
    model,
    r=4,
    lora_alpha=16,
    lora_dropout=0,
    use_gradient_checkpointing=True,
    random_state=42,
    max_seq_length=max_seq_length,
)

"""

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-PCIE-40GB. Num GPUs = 1. Max memory: 39.495 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu118. CUDA: 8.0. CUDA Toolkit: 11.8. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 8/8 [00:09<00:00,  1.13s/it]


'\nmodel = FastLanguageModel.get_peft_model(\n    model,\n    r=4,\n    lora_alpha=16,\n    lora_dropout=0,\n    use_gradient_checkpointing=True,\n    random_state=42,\n    max_seq_length=max_seq_length,\n)\n\n'

In [7]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_data,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 300,
        learning_rate = 1e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)



trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n"
)

Unsloth: Tokenizing ["text"] (num_proc=2): 100%|██████████| 201/201 [00:00<00:00, 205.86 examples/s]
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Map (num_proc=128): 100%|██████████| 201/201 [00:01<00:00, 142.68 examples/s]


In [8]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nunknownspeaker: I see her in the waiting room. Hang on, just. OK.\njonnoalcaro: We need a motion to approve the consent agenda. I move that we approve the consent agenda. Is there a second?\nunknownspeaker: I second. Moved by Mr. Alcaro, seconded by Ms. Lita, we approve the consent agenda. Any discussion on the motion? If not, Ms. Johnston, could you please call the roll? Dr. Acuff. Yes. Ms. Colson. Yes. Mr. Oberg. Yes. Ms. Lee. Yes. Ms. Osborne. Yes. Mr. Alcaro. Yes. Mr. Page. Yes. Motion carried.\ngrahampaige: Motion carries. All right. Thank you. Our next item on the agenda is a spotlight on education. And we'll be led tonight in that by Dr. Acuff. Dr. Acuff.<|eot_id|>\n\n<|start_header_id|>assistant<|end_header_id|>\n\nkateacuff: Thank you, Chair Page. I apologize in advance if I freeze. I've sent a copy of it to Mr. Elkaro, so he might, if I freeze all the way out, then maybe he can pick up the pieces. 

In [9]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                                                                                                                                                              kateacuff: Thank you, Chair Page. I apologize in advance if I freeze. I've sent a copy of it to Mr. Elkaro, so he might, if I freeze all the way out, then maybe he can pick up the pieces. But I think it was on May 7, 2020, nearly 40 school board meetings ago, the acronym COVID for students and families, our staff, and our broader community has been the board and for our entire school division. The good news is that while there has been some 185 confirmed cases of COVID among our students, staff, and school, there have been few cases of virus transmission inside our schools. Today, nearly three out of four of our eligible students are fully vaccinated, and in the first few weeks of our vaccination clinics last spring, nearly 2,000 of our employees received vaccines. In little over one week, we will 

In [9]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 201 | Num Epochs = 6 | Total steps = 300
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 4 x 1) = 4
 "-____-"     Trainable parameters = 51,773,440/70,000,000,000 (0.07% trained)
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f87b6988d10>>
Traceback (most recent call last):
  File "/work/users/s/m/smerrill/.conda/envs/unsloth_env/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 

KeyboardInterrupt

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f87b6988d10>>
Traceback (most recent call last):
  File "/work/users/s/m/smerrill/.conda/envs/unsloth

In [10]:
for i in train_data['text']:
    print(i)
    print('-------------------------')

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

katrinacallser: So is that our target. just at the beginning of February. I'm wondering if that's the target.
patrickmclaughlin: That was the original target when we were using December 10th as a decision point. With the shift to January 21st as a decision point, we're not quite sure we can get things turned around by February 1st to have things ready to go if we got the green light to go back. So if there's feedback from the board about that timeline, please send it my way and we can discuss that in cabinet as well.
katrinacallser: Yeah, I guess my little piece of feedback is just, I guess maybe I should talk with you because I'm just kind of curious, not curious, but I'm kind of concerned, I guess, about backing off of our timeline. I get the rationale, but then to say that it's not even going to line up to when we're having a semester shift in high school is, I guess, a little, and then to hear that the city is pushing for e

## Single Evaluation

In [11]:
train_completion_data[0]['completion']

" Thank you, Chair Page. Last month, we had the wonderful opportunity to hear from Baker Butler educators about the nationally recognized gains they have made in closing the achievement gap at their school. Tonight we're focusing on yet another opportunity gap in our schools and in schools across our nation, the percentage of students of color who enter STEM related fields. 45 years ago, an organization was established to address this issue, the National Society of Black Engineers. Its mission was ambitious and vital to the collective prosperity of every community in the United States. The society was organized to promote academic excellence, provide scholarships, train leaders, and expand access to professional careers. A few years ago, the society broadened its charter to include high school chapters, and one of the first chapters in Central Virginia was established at Albemarle High School. Membership is open to students from all of our middle school and high schools, regardless of 

In [12]:
train_completion_data[0]['prompt']

"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\ngrahampaige: And our student reps, tonight we have both our regular rep along with an alternate that had not had a chance to sit in on any of our meetings. So first, Will Trout.\nunknownspeaker: Will Trout, student representative. Okay, and tell us again where your school, Will. Yes, Monticello. Okay, and Elijah Witt. Elijah Witt, alternate student representative at Western Albemarle.\ngrahampaige: Okay, thank you. And Elijah, we are very pleased to have you with us tonight, and you'll also be appearing at our December meetings, so thanks so much. We will now entertain a moment, I mean a motion rather, for the approval of the agenda.\nunknownspeaker: I move that we approve the agenda. Second. Second.\ngrahampaige: OK, moved by Ms. DeAlcaro, second by Dr. Acuff, that we approve the agenda. Ms. Johnston.\nunknownspeaker: Ms. Osborne. Yes. Ms. Galston. Yes. Dr. Acuff? Yes. Mr. Alcaro? Yes. Mr. Oberg? Yes. Ms. Lee? Yes. Mr. Page

In [15]:
input_text = train_completion_data[5]['prompt']

inputs = tokenizer(
    input_text,  # your full prompt string
    return_tensors="pt"
).to(model.device)

# Create a streamer to print output as it's generated (optional)
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Generate a response
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
    use_cache=True,
    temperature=1.5,
    top_p=0.9  # (use `top_p`, not `min_p`)
)

 It strikes me, Dr. Collins, that we ought to be prepared for that to be sort of, there's a possibility, the extent that we reopen, there's going to be, that could be a growing issue. Saying as people get more comfortable, hey, look, it's been three weeks and I don't have any issues. Maybe I want to go back to school. I think that that may just be, that may be, I just hope we're prepared for that.
judyle: And that change could go either way, right? What about if the person is in and decides to go to virtual? How long is the


### Full evaluation

In [8]:
def generate_texts(model, tokenizer, dataset, smax_new_tokens=128, temperature=1.5, top_p=0.9):
    generated_texts = []
    reference_texts = []

    for example in tqdm(dataset):
        input_text = example['prompt']

        inputs = tokenizer(
            input_text,
            return_tensors="pt"
        ).to(model.device)

        input_length = inputs.input_ids.shape[1]

        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            use_cache=True,
            temperature=temperature,
            top_p=top_p
        )

        generated_tokens = outputs[0]
        new_tokens = generated_tokens[input_length:]

        decoded_generation = tokenizer.decode(new_tokens, skip_special_tokens=True)

        print(decoded_generation)
        generated_texts.append(decoded_generation.strip())
        reference_texts.append(example['completion'].strip())

    return generated_texts, reference_texts

In [12]:
from tqdm import tqdm
max_new_tokens = 100
generated_texts, reference_texts = generate_texts(model, tokenizer, train_completion_data, smax_new_tokens=128, temperature=1.5, top_p=0.9)

  0%|          | 0/240 [00:06<?, ?it/s]

KeyboardInterrupt


KeyboardInterrupt



In [9]:
from tqdm import tqdm
max_new_tokens = 128
generated_texts, reference_texts = generate_texts(model, tokenizer, test_data, smax_new_tokens=128, temperature=1.5, top_p=0.9)

  5%|▌         | 1/20 [00:12<03:58, 12.58s/it]

Patrick: Sure can, yep. 
(Patrick retrieves the map and displays it for everyone to see)
Mr. Oberg: And Mr. Oberg, that comprises the neighborhoods of Gray Rock, Wayland's Grant and Bargaman Park in this larger area of Western Ridge, Wickham Pond. 
(Patrick is not sharing the map as he thought, so he apologizes and continues)
Mr. Oberg: Oops, sorry, I thought it was sharing, but it's not. So is it coming through now? 
Mr. Oberg: Yes. Great. So that comprises the neighborhoods of Gray Rock here


 10%|█         | 2/20 [00:24<03:40, 12.26s/it]

Thank you, Ms. Hicks. I really appreciate your coming and being a part of this. So thank you so much. I got a lot of notes on this. And Mr. Kearse, could I just ask you one quick question for the benefit of the board and of the public? Would you explain about what the Parent Resource Center is and why we're stepping away from that and how that's going to be, that void might get filled in? Sure.

So the Parent Resource Center is, I mean, it's a beautiful title, right? It's a center, a person provides services through the center for parents in our


 15%|█▌        | 3/20 [00:37<03:32, 12.48s/it]

...

lourenmacle.: So due to the increase we've seen in our English language learner (ELL) population, and a continued projected increase, we are seeking to add a FTE (Fundamental Teacher Expense) to provide additional support in our urban ring secondary schools, specifically to cater to the needs of newcomer students with limited native language literacy or interrupted formal schooling.

Additionally, someone asked a question: "So I'll go ahead and ask my question here. Again, just doing the head math, it seems as though adding one FTE still puts us below the ratio that we were at before last year. Is that correct?"

The response


 95%|█████████▌| 19/20 [02:54<00:10, 10.64s/it]

Thank you for the discussion. I will now provide a summary of the key points discussed:

1. **Budget**: The budget for next year is projected to be $7.21 million, with an increase of 4.1% in operating funds. However, due to a mistake in earlier projections, the actual increase is expected to be 4.7%.

2. **Enrollment**: The projected enrollment for K-12 students is 721. However, due to an error in the previous response, it was mentioned that another figure had been exceeded. To clarify, the actual enrollment is expected to be close to 721, not


100%|██████████| 20/20 [03:02<00:00,  9.12s/it]

 No, I'm fine, thank you. My next meeting is actually an Executive Cabinet meeting to further discuss the financial situation. With that, we're done for this evening. Thank everyone for your questions, for helping us prepare and improve. Have a good night and thank you, Mr. Chairman, for your chairing this evening.


In [10]:
import evaluate  # instead of datasets.load_metric

# Initialize metrics
bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")


In [11]:
def compute_metrics(generated_texts, reference_texts):

    # Compute metrics
    bleu_score = bleu.compute(predictions=generated_texts, references=[[r] for r in reference_texts])
    rouge_score = rouge.compute(predictions=generated_texts, references=reference_texts)
    bertscore_result = bertscore.compute(predictions=generated_texts, references=reference_texts, lang="en")

    # Average BERTScore F1
    avg_bertscore_f1 = sum(bertscore_result['f1']) / len(bertscore_result['f1'])
            
    return bleu_score, rouge_score, bertscore_result, avg_bertscore_f1

In [12]:
bleu_score, rouge_score, bertscore_result, avg_bertscore_f1 = compute_metrics(generated_texts, reference_texts)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
bleu_score, rouge_score, bertscore_result, avg_bertscore_f1

({'bleu': 0.006077129561234383,
  'precisions': [0.15736934820904286,
   0.011876484560570071,
   0.0012012012012012011,
   0.0006075334143377885],
  'brevity_penalty': 1.0,
  'length_ratio': 1.5724838411819022,
  'translation_length': 1703,
  'reference_length': 1083},
 {'rouge1': np.float64(0.1541466945728513),
  'rouge2': np.float64(0.017248572697572115),
  'rougeL': np.float64(0.10011529596140455),
  'rougeLsum': np.float64(0.10759896171220223)},
 {'precision': [0.8012092113494873,
   0.8241617679595947,
   0.689644455909729,
   0.8287801742553711,
   0.833304762840271,
   0.828080415725708,
   0.847261905670166,
   0.8211007118225098,
   0.8315057754516602,
   0.8339003324508667,
   0.0,
   0.8351423144340515,
   0.8233726620674133,
   0.8133792877197266,
   0.8384840488433838,
   0.8493857383728027,
   0.8207684755325317,
   0.8209556341171265,
   0.8152289986610413,
   0.8355612754821777],
  'recall': [0.8248466849327087,
   0.8207997679710388,
   0.7795955538749695,
   0.824158

In [20]:
gc.collect()

2271